In [4]:
#2 # 2 in spark
# Create a new Spark notebook called labd.ipynb. Write (or copy and edit) Spark code to 
# set up the Spark session. Make sure your Spark session supports Minio access and 
# include the hadoop-aws Spark Jar package. Provide a screenshot of your code and the output.

import pyspark
from pyspark.sql import SparkSession
print("hello")

hello


In [5]:
# MINIO CONFIGURATION
s3_host = "minio"
s3_url = f"http://{s3_host}:9000"
s3_key = "minio"
s3_secret = "SU2orange!"
s3_bucket = "labd"
print(s3_url)

http://minio:9000


In [6]:
# Spark init
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.1.2")\
    .config("spark.hadoop.fs.s3a.endpoint", s3_url) \
    .config("spark.hadoop.fs.s3a.access.key", s3_key) \
    .config("spark.hadoop.fs.s3a.secret.key", s3_secret) \
    .config("spark.hadoop.fs.s3a.fast.upload", True) \
    .config("spark.hadoop.fs.s3a.path.style.access", True) \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()
sc = spark.sparkContext
print(s3_bucket)
#sc.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-76d7fb8b-0c74-4ffe-aa6f-d60152a890b3;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.1.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.271 in central
:: resolution report :: resolve 302ms :: artifacts dl 7ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.271 from central in [default]
	org.apache.hadoop#hadoop-aws;3.1.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------

labd


In [7]:
# 3
# Write Spark code to load logs from Minio labd/logs into a dataframe logs1 using 
# spark.read.text. Print the schema and show 10 rows from the DataFrame. 
# Screenshot the code and output.
s3_bucket ="labd"
print("hello")
log_in = f"s3a://{s3_bucket}/logs/*.log"
print(log_in)
logs1 = spark.read.text(log_in)
logs1.show()


hello
s3a://labd/logs/*.log


23/01/26 22:58:36 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+--------------------+
|               value|
+--------------------+
|#Software: Micros...|
|       #Version: 1.0|
|#Date: 2016-02-11...|
|#Fields: date tim...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
+--------------------+
only showing top 20 rows



In [8]:
#4 
# We need to remove the rows with # in front of them, as these are comments in 
# the web server log files. Use the filter() function to do this, and save the 
# results into DataFrame logs2. Show the code and output in your screenshot.

logs2 = logs1.filter(~logs1.value.startswith("#"))
logs2.show()


+--------------------+
|               value|
+--------------------+
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
|2016-02-11 17:16:...|
+--------------------+
only showing top 20 rows



In [9]:
#5
# Write back your logs2 to Minio. Use the text format and call the file 
# logs-no-header. Include a screenshot of the code and output.

logs_out = f"s3a://{s3_bucket}/logs/logs_no_header"
logs2.write.text(logs_out)
print(logs_out)

23/01/26 23:00:08 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
23/01/26 23:00:09 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


s3a://labd/logs/logs_no_header


In [12]:
#6
# Read in the logs-no-header, this time using csv to delimit on a space into logs3. Add headers 
# (date,time, serverip, method, uri, querystring, port, username, clientip, useragent, 
# referrer, statuscode), and provide an output of the schema and the first couple of rows 
# from the dataframe itself in the screenshot.
logs3 = spark.read.option("sep", " ").option("inferSchema", True).csv(logs_out)
logs3.printSchema()
#.toDF("date","time", "serverip", "method", "uri", "querystring", "port", "username", "clientip", "useragent","referrer", "statuscode", _, _, _)\
#.select("date","time", "serverip", "method", "uri", "querystring", "port", "username", "clientip", "useragent", "referrer", "statuscode")
#logs3.show()
#logs3.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)
 |-- _c13: integer (nullable = true)
 |-- _c14: integer (nullable = true)



In [13]:
logs4 = spark.read.option("sep", " ").option("inferSchema", True).csv(logs_out)\
.toDF("date","time", "serverip", "method", "uri", "querystring", "port", "username", "clientip", "useragent","referrer", "statuscode", _, _, _)\
.select("date","time", "serverip", "method", "uri", "querystring", "port", "username", "clientip", "useragent", "referrer", "statuscode")
logs4.show()


+----------+--------+--------------+------+--------------------+-----------+----+--------+-----------+--------------------+--------------------+----------+
|      date|    time|      serverip|method|                 uri|querystring|port|username|   clientip|           useragent|            referrer|statuscode|
+----------+--------+--------------+------+--------------------+-----------+----+--------+-----------+--------------------+--------------------+----------+
|2016-02-11|17:16:13|128.230.247.37|   GET|                   /|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|                   -|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Content/jquery-u...|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|http://group0.ist...|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Plugins/Widgets....|          -|  80|       -|215.82.23.2|Mozilla/5.0+(Wind...|http://group0.ist...|       200|
|2016-02-11|17:16:13|128.230.247.37|   GET|/Plugins/Widgets....|

In [14]:
#7 
# Let's handle the IP address lookup data. Write Spark code to load the iplookup.json 
# file from Minio into the DataFrame ips1. Show the first 10 rows and print 
# the schema for a screenshot to include code and output.

iplookup_in = f"s3a://{s3_bucket}/iplookup/iplookup.json"
ips1 = spark.read.option("multiline", True).json(iplookup_in)
ips1.printSchema()
ips1.show(10)

root
 |-- geography: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- state: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- location: struct (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- lng: double (nullable = true)

+--------------------+---------------+--------------------+
|           geography|             ip|            location|
+--------------------+---------------+--------------------+
|   {Dulles, USA, VA}|  172.189.252.8|{38.955855, -77.4...|
| {Columbus, USA, OH}|    215.82.23.2|{39.961176, -82.9...|
|{Cleveland, USA, OH}|    98.29.25.44|{41.49932, -81.69...|
| {Freeport, USA, NY}|  68.199.40.156|{40.657602, -73.5...|
|{Salt Lake City, ...|155.100.169.152|{40.760779, -111....|
|   {Dallas, USA, TX}|   38.68.15.223|{32.776664, -96.7...|
|    {Tampa, USA, FL}|   70.209.14.54|{27.950575, -82.4...|
|{Arlington, USA, VA}|   74.111.6.173|{38.87997, -77.10...|
| {Syracuse,

In [19]:
#8 
# We need to flatten the nested JSON data. Use the select() function with dot notation to 
# do this, saving the DataFrame as ips2. Provide a screenshot of the schema and 
# output of the first few rows.

#ips2 = ips2.select("ip", "geography.city","geography.state","geography.country", "location.lat", "location.lng")
ips2 = ips1.select("ip", "geography.city","geography.state","geography.country", ips1.location.lng, ips1.location.lng.alias("lng"))
ips2.show()

+---------------+--------------+-----+-------+------------+-----------+
|             ip|          city|state|country|location.lng|        lng|
+---------------+--------------+-----+-------+------------+-----------+
|  172.189.252.8|        Dulles|   VA|    USA|  -77.447819| -77.447819|
|    215.82.23.2|      Columbus|   OH|    USA|  -82.998794| -82.998794|
|    98.29.25.44|     Cleveland|   OH|    USA|  -81.694361| -81.694361|
|  68.199.40.156|      Freeport|   NY|    USA|  -73.583184| -73.583184|
|155.100.169.152|Salt Lake City|   UT|    USA| -111.891047|-111.891047|
|   38.68.15.223|        Dallas|   TX|    USA|  -96.796988| -96.796988|
|   70.209.14.54|         Tampa|   FL|    USA|  -82.457178| -82.457178|
|   74.111.6.173|     Arlington|   VA|    USA|   -77.10677|  -77.10677|
|128.230.122.180|      Syracuse|   NY|    USA|  -76.147424| -76.147424|
|128.122.140.238|      New York|   NY|    USA|  -74.005941| -74.005941|
| 56.216.127.219|       Raleigh|   NC|    USA|  -78.638179| -78.

In [21]:
#9 
# Now join the two DataFrames together on their business key, making the new DataFrame comb1. 
# Provide a schema and sample of the first few rows in your screenshot.
comb1 = ips2.join(logs4, on = ips2.ip == logs4.clientip, how="inner")
comb1.printSchema()
comb1.show(75)

root
 |-- ip: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- location.lng: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- serverip: string (nullable = true)
 |-- method: string (nullable = true)
 |-- uri: string (nullable = true)
 |-- querystring: string (nullable = true)
 |-- port: integer (nullable = true)
 |-- username: string (nullable = true)
 |-- clientip: string (nullable = true)
 |-- useragent: string (nullable = true)
 |-- referrer: string (nullable = true)
 |-- statuscode: integer (nullable = true)

+---------------+--------+-----+-------+------------+----------+----------+--------+--------------+------+--------------------+-----------+----+--------+---------------+--------------------+--------------------+----------+
|             ip|    city|state|country|location.lng|       lng|      date|  

In [22]:
#10 
# Write the comb1 DataFrame in parquet format back to Minio in the 
# folder cleaned-logs. Again, show evidence the code ran and the file was created.
cleaned_logs_out = f"s3a://{s3_bucket}/iplookup/cleaned-logs"
comb1.write.mode("Overwrite").parquet(cleaned_logs_out)